In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [2]:
sleep_time = 1 # Đơn vị: giây

In [3]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

In [39]:
re = requests.get('https://api-v2.soundcloud.com/users/630253005/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

In [40]:
json_pydata = json.loads(re.text)
json_pydata

{'collection': [{'avatar_url': 'https://i1.sndcdn.com/avatars-Z9f2ZHjqZ2RSas7J-KXHEdA-large.jpg',
   'city': '',
   'comments_count': 283,
   'country_code': None,
   'created_at': '2016-04-06T15:07:33Z',
   'creator_subscriptions': [{'product': {'id': 'creator-pro-unlimited'}}],
   'creator_subscription': {'product': {'id': 'creator-pro-unlimited'}},
   'description': 'High quality free to use music for your monetized UGC (User Generated Content) on platforms like YouTube, Instagram, Twitch, and Facebook.\n\nBefore using any of the music on our media platforms, we highly recommend you read the FAQs on (https://freetousemusic.com/), to better understand the whole world of free to use music and how it works.\n\nFree To Use Music is part of the official Free To Use Network providing free to use content for creators. Free To Use Network is a collection of media platforms owned by the Danish company Free To Use ApS.\n\nDo you want your music to be discovered by millions of people?\nPlease 

In [41]:
for user in json_pydata['collection']:
    print(user['id'])

217441590
324531068
525378972
9855085
732907996
311417284
331219188
733024996
99914158
348220670


In [42]:
a = set([1])
a

{1}

In [43]:
a

{1}

In [44]:
def get_user_id(init_id, limit, client_id):
    
    list_user_id = [init_id]# User id list
    set_user_id = set(list_user_id) # User id set
    index = 0 # Used for tracing more users
    
    while len(list_user_id) < limit:
        # Không còn user id để trace
        if index == len(list_user_id):
            break
        
        # Lấy dữ liệu với url tương ứng
        url = f'https://api-v2.soundcloud.com/users/{list_user_id[index]}/followings?client_id={client_id}'
        print('Get:', url)
        re = requests.get(url)
        
        # Vòng lặp kiểm tra request đã sẵn sàng để crawl chưa
        canGet = False
        while not canGet:
            # Nếu request không được thì sleep rồi request lại
            if not re.ok:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Not ok')
                continue
            
            # Kiểm tra request có bị timeout
            # Nếu có sleep rồi request lại
            if re.status_code == 504:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Time out')
                continue
                
            if re.status_code == 429:
                print('Exceeds an API rate limit')
                return list_user_id
                
            # Đổi Json string thành python dictionary
            json_pydata = json.loads(re.text)
            
            # Request sẵn sàng để crawl
            canGet = True
            
        print('Pass:', url)
            
        # Thêm user id
        for user in json_pydata['collection']:
            if user['id'] not in set_user_id:
                set_user_id.add(user['id'])
                list_user_id.append(user['id'])
        
        index = index + 1
        
    return list_user_id

In [45]:
list_user_id = get_user_id(630253005, 1000, '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

Get: https://api-v2.soundcloud.com/users/630253005/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/630253005/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/217441590/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/217441590/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/324531068/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/324531068/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/525378972/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/525378972/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/9855085/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/9855085/fol

In [46]:
np.savetxt('user_id.txt', list_user_id, fmt='%d')

In [6]:
def get_user_profile(user_id_file, client_id):
    
    list_user_id = np.loadtxt(user_id_file, dtype=int) # User id list
    
    # Initialize track dataframe
    user = pd.DataFrame(columns = ['city', 'created_at', 'followers_count', 'followings_count', 'id', 
                                   'playlist_count','track_count','verified', 'username', 
                                   'likes_count', 'permalink_url'])
    
    # Attributes in the track dataframe 
    attributes = ['city', 'created_at', 'followers_count', 'followings_count', 'id', 
                    'playlist_count','track_count','verified', 'username', 
                 'likes_count', 'permalink_url']

    for user_id in list_user_id:    
        # Get request with corresponding user_id
        url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
        print('Get:', url)
        print('Status: Doing')
        re = requests.get(url)
        
        # Loop for checking if can make a request
        canGet = False
        while not canGet:
            # Sleep then make the request again
            if not re.ok:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Status: Not ok')
                continue
            
            # Check request timeout
            # If it is timeout, sleep then make the request again
            if re.status_code == 504:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Status: Timeout')
                continue
                
            if re.status_code == 429:
                print('Status: Exceeds an API rate limit')
                return user
                
            # Convert Json string into python dictionary
            json_pydata = json.loads(re.text)
            
            # Request is done
            canGet = True
            
        print('Status: Done')
            
        temp = {} # Temporary dictionary for storing the current playlist record
            
        # Get corresponding attributes into temp
        for key in attributes:
            temp[key] = json_pydata[key]
            
        # Append the record into the dataframe
        user = user.append(temp, ignore_index=True)
        
    return user

In [7]:
user = get_user_profile('user_id.txt', '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

Get: https://api-v2.soundcloud.com/users/630253005?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/217441590?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/324531068?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/525378972?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/9855085?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/732907996?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/311417284?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/331219188?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com

KeyboardInterrupt: 